<a href="https://colab.research.google.com/github/firecutter/A.I-PROJECTS/blob/main/perfil_de_comprador_de_imovel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modulo de geração de dados mocks

In [57]:
import random
import pandas as pd
from faker import Faker

fake = Faker('pt_BR')


regioes_df = {
    "Asa Sul": "Centro",
    "Asa Norte": "Centro",
    "Lago Norte": "Zona Nobre",
    "Lago Sul": "Zona Nobre",
    "Taguatinga": "Zona Metropolitana",
    "Ceilândia": "Zona Metropolitana",
    "Samambaia": "Zona Metropolitana",
    "Guará": "Zona Metropolitana",
    "Planaltina": "Entorno",
    "Sobradinho": "Entorno"
}


def classificar_faixa_etaria(idade):
    if idade < 30:
        return "jovem_adulto"
    elif idade < 60:
        return "adulto"
    else:
        return "idoso"

def gerar_renda_por_faixa(faixa):
    if faixa == "jovem_adulto":
        return random.randint(2000, 7000)
    elif faixa == "adulto":
        return random.randint(4000, 15000)
    else:
        return random.randint(2500, 9000)

def preferencia_por_faixa(faixa):
    if faixa == "jovem_adulto":
        return random.choice(["Kitnet", "Apartamento"])
    elif faixa == "adulto":
        return random.choice(["Apartamento", "Casa"])
    else:
        return random.choice(["Casa", "Apartamento"])


def gerar_estado_civil():
    return random.choice(["Solteiro", "Casado", "Divorciado"])


def gerar_dados_fake(num_registros: int):
    dados = []

    regioes = list(regioes_df.keys())

    for _ in range(num_registros):
        nome = fake.name()
        idade = random.randint(18, 80)
        faixa = classificar_faixa_etaria(idade)
        renda_mensal = gerar_renda_por_faixa(faixa)
        tipo_imovel = preferencia_por_faixa(faixa)
        localizacao = random.choice(regioes)
        regiao_classificada = regioes_df[localizacao]
        estado_civil = gerar_estado_civil()  # Gerando estado civil aleatório
        status = random.choice(["Ativo", "Inativo"])

        dados.append([
            nome,
            idade,
            faixa,
            renda_mensal,
            tipo_imovel,
            localizacao,
            regiao_classificada,
            estado_civil,  # Adicionando estado civil
            status
        ])


    df = pd.DataFrame(dados, columns=[
        "nome",
        "idade",
        "perfil_faixa_etaria",
        "renda_mensal",
        "tipo_imovel_desejado",
        "localizacao",
        "regiao_df",
        "estado_civil",  # Nova coluna
        "status"
    ])

    return df

if __name__ == "__main__":
    num_registros = 1000
    df_gerado = gerar_dados_fake(num_registros)
    df_gerado.to_csv("dados_fake.csv", index=False)
    print(f"{num_registros} registros gerados com contexto de Brasília e salvos em 'dados_fake.csv'.")



1000 registros gerados com contexto de Brasília e salvos em 'dados_fake.csv'.


In [65]:


from google.colab import files
files.download('dados_fake.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 Frontend Streamlit e Análise

In [69]:
import pandas as pd
import plotly.express as px


def carregar_dados(arquivo):
    return pd.read_csv(arquivo)


def analisar_dados(dados):
    print("Dados carregados:")
    print(dados.head(50))
    print(f"Total de registros: {len(dados)}")


    if 'renda_mensal' in dados.columns:
        faixa_renda = (2000, 15000)
        filtrado = dados[(dados['renda_mensal'] >= faixa_renda[0]) & (dados['renda_mensal'] <= faixa_renda[1])]
        print(f"Registros na faixa de renda selecionada: {len(filtrado)}")
        print(filtrado.head(20))


        bins = list(range(2000, 16000, 1000))
        labels = [f"{bins[i]} a {bins[i+1]-1}" for i in range(len(bins)-1)]
        filtrado['faixa_renda'] = pd.cut(filtrado['renda_mensal'], bins=bins, labels=labels, right=False)

        renda_faixas_contagem = filtrado['faixa_renda'].value_counts().sort_index().reset_index()
        renda_faixas_contagem.columns = ['Faixa de Renda', 'Quantidade']


        fig_renda_faixas = px.bar(
            renda_faixas_contagem,
            x='Faixa de Renda',
            y='Quantidade',
            title="Distribuição por Faixas de Renda Mensal",
            color='Faixa de Renda',
            color_discrete_sequence=px.colors.sequential.Blues
        )
        fig_renda_faixas.update_layout(
            xaxis_title='Faixa de Renda (R$)',
            yaxis_title='Quantidade de Pessoas',
            template='plotly_dark',
            bargap=0.3,
            showlegend=False
        )
        fig_renda_faixas.show()


        if 'idade' in filtrado.columns:
            fig_idade = px.histogram(
                filtrado,
                x='idade',
                nbins=20,
                title="Distribuição de Idade",
                color_discrete_sequence=['#636EFA']
            )
            fig_idade.update_layout(
                xaxis_title='Idade',
                yaxis_title='Quantidade',
                bargap=0.3,
                template='plotly_dark'
            )
            fig_idade.show()


        if 'tipo_imovel_desejado' in filtrado.columns:
            tipo_imovel_counts = filtrado['tipo_imovel_desejado'].value_counts().reset_index()
            tipo_imovel_counts.columns = ['tipo_imovel_desejado', 'count']

            fig_tipo = px.bar(
                tipo_imovel_counts,
                x='tipo_imovel_desejado',
                y='count',
                title="Distribuição das Preferências de Imóveis",
                color='tipo_imovel_desejado',
                color_discrete_sequence=px.colors.sequential.Plasma
            )
            fig_tipo.update_layout(
                xaxis_title='Tipo de Imóvel',
                yaxis_title='Quantidade',
                bargap=0.3,
                template='plotly_dark',
                showlegend=False
            )
            fig_tipo.show()


if __name__ == "__main__":
    arquivo = 'dados_fake.csv'
    dados = carregar_dados(arquivo)
    analisar_dados(dados)



Dados carregados:
                           nome  idade perfil_faixa_etaria  renda_mensal  \
0   Sr. Carlos Eduardo Nogueira     80               idoso          8642   
1            Valentina Cassiano     73               idoso          7835   
2              Fernando Freitas     54              adulto         14560   
3     Srta. Maria Flor Carvalho     70               idoso          7674   
4       Luiz Henrique Aparecida     52              adulto          8620   
5                Catarina Gomes     52              adulto          7798   
6           Vitor Hugo Mendonça     48              adulto         11761   
7                Benjamin Ramos     34              adulto          5992   
8               Bryan das Neves     31              adulto         14156   
9           Maria Luísa Fonseca     46              adulto          7843   
10                 Ana Caldeira     52              adulto          8214   
11              Manuela Peixoto     32              adulto          57

Filtros Interativos e Geração de Gráficos

In [68]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


def carregar_dados(arquivo):
    return pd.read_csv(arquivo)


def analisar_dados(dados):

    print("Dados carregados:")
    print(dados.head(50))

    print(f"Total de registros: {len(dados)}")


    if 'renda_mensal' in dados.columns:
        faixa_renda = (2000, 15000)
        filtrado = dados[(dados['renda_mensal'] >= faixa_renda[0]) & (dados['renda_mensal'] <= faixa_renda[1])]
        print(f"Registros na faixa de renda selecionada: {len(filtrado)}")
        print(filtrado.head(20))


        fig_renda = px.histogram(filtrado, x='renda_mensal', nbins=30, title="Distribuição da Renda Mensal",
                                 color_discrete_sequence=['#4CAF50'])


        fig_renda.update_traces(texttemplate='%{y}', textposition='outside', marker=dict(line=dict(width=2, color='black')))
        fig_renda.update_layout(
            xaxis_title="Renda Mensal (R$)",
            yaxis_title="Quantidade",
            title_x=0.5,
            title_font_size=18,
            title_font_color="#2F4F4F",
            template="plotly_white",
            showlegend=False
        )
        fig_renda.show()


        if 'idade' in filtrado.columns:
            fig_idade = px.histogram(filtrado, x='idade', nbins=20, title="Distribuição de Idade",
                                     color_discrete_sequence=['#2196F3'])
            fig_idade.update_traces(texttemplate='%{y}', textposition='outside', marker=dict(line=dict(width=2, color='black')))
            fig_idade.update_layout(
                xaxis_title="Idade (anos)",
                yaxis_title="Quantidade",
                title_x=0.5,
                title_font_size=18,
                title_font_color="#2F4F4F",
                template="plotly_white",
                showlegend=False
            )
            fig_idade.show()


        if 'tipo_imovel_desejado' in filtrado.columns:
            tipo_imovel_counts = filtrado['tipo_imovel_desejado'].value_counts().reset_index()
            tipo_imovel_counts.columns = ['tipo_imovel_desejado', 'count']

            fig_tipo = px.bar(tipo_imovel_counts,
                              x='tipo_imovel_desejado', y='count',
                              labels={'tipo_imovel_desejado': 'Tipo de Imóvel', 'count': 'Quantidade'},
                              title="Distribuição das Preferências de Imóveis",
                              color='count', color_continuous_scale='Blues')

            fig_tipo.update_traces(texttemplate='%{y}', textposition='outside', marker=dict(line=dict(width=2, color='black')))
            fig_tipo.update_layout(
                xaxis_title="Tipo de Imóvel",
                yaxis_title="Quantidade",
                title_x=0.5,
                title_font_size=18,
                title_font_color="#2F4F4F",
                template="plotly_white",
                showlegend=False
            )
            fig_tipo.show()


if __name__ == "__main__":
    arquivo = 'dados_fake.csv'
    dados = carregar_dados(arquivo)
    analisar_dados(dados)




Dados carregados:
                           nome  idade perfil_faixa_etaria  renda_mensal  \
0   Sr. Carlos Eduardo Nogueira     80               idoso          8642   
1            Valentina Cassiano     73               idoso          7835   
2              Fernando Freitas     54              adulto         14560   
3     Srta. Maria Flor Carvalho     70               idoso          7674   
4       Luiz Henrique Aparecida     52              adulto          8620   
5                Catarina Gomes     52              adulto          7798   
6           Vitor Hugo Mendonça     48              adulto         11761   
7                Benjamin Ramos     34              adulto          5992   
8               Bryan das Neves     31              adulto         14156   
9           Maria Luísa Fonseca     46              adulto          7843   
10                 Ana Caldeira     52              adulto          8214   
11              Manuela Peixoto     32              adulto          57

In [60]:
import pandas as pd
from faker import Faker

fake = Faker('pt_BR')


df = pd.read_csv("dados_fake.csv")

def gerar_bonecos_por_perfil(df):
    perfis = df['perfil_faixa_etaria'].unique()

    print("\n--- Bonecos dos Perfis ---\n")

    for perfil in perfis:
        grupo = df[df['perfil_faixa_etaria'] == perfil]

        idade_media = int(grupo['idade'].mean())
        renda_media = int(grupo['renda_mensal'].mean())
        imovel_mais_desejado = grupo['tipo_imovel_desejado'].mode()[0]
        regiao_mais_comum = grupo['localizacao'].mode()[0]

        nome_ficticio = fake.name()

        print(f"👤 Perfil: {perfil.replace('_', ' ').title()}")
        print(f"   Nome: {nome_ficticio}")
        print(f"   Idade média: {idade_media} anos")
        print(f"   Renda média: R$ {renda_media}")
        print(f"   Tipo de imóvel desejado: {imovel_mais_desejado}")
        print(f"   Região predominante: {regiao_mais_comum}")
        print("-" * 50)
s
gerar_bonecos_por_perfil(df)



--- Bonecos dos Perfis ---

👤 Perfil: Adulto
   Nome: Carolina Casa Grande
   Idade média: 44 anos
   Renda média: R$ 9532
   Tipo de imóvel desejado: Apartamento
   Região predominante: Lago Sul
--------------------------------------------------
👤 Perfil: Jovem Adulto
   Nome: Maria Vitória Nunes
   Idade média: 23 anos
   Renda média: R$ 4518
   Tipo de imóvel desejado: Kitnet
   Região predominante: Asa Sul
--------------------------------------------------
👤 Perfil: Idoso
   Nome: Fernanda Cassiano
   Idade média: 69 anos
   Renda média: R$ 5743
   Tipo de imóvel desejado: Apartamento
   Região predominante: Guará
--------------------------------------------------


In [61]:
import pandas as pd
from faker import Faker

fake = Faker('pt_BR')

df = pd.read_csv("dados_fake.csv")

def gerar_bonecos_por_perfil(df):
    perfis = df['perfil_faixa_etaria'].unique()

    print("\n--- Bonecos dos Perfis Gerais ---\n")

    for perfil in perfis:
        grupo = df[df['perfil_faixa_etaria'] == perfil]

        idade_media = int(grupo['idade'].mean())
        renda_media = int(grupo['renda_mensal'].mean())
        imovel_mais_desejado = grupo['tipo_imovel_desejado'].mode()[0]
        regiao_mais_comum = grupo['localizacao'].mode()[0]

        nome_ficticio = fake.name()

        print(f"👤 Perfil: {perfil.replace('_', ' ').title()}")
        print(f"   Nome: {nome_ficticio}")
        print(f"   Idade média: {idade_media} anos")
        print(f"   Renda média: R$ {renda_media}")
        print(f"   Tipo de imóvel desejado: {imovel_mais_desejado}")
        print(f"   Região predominante: {regiao_mais_comum}")
        print("-" * 50)


def perfil_ceilandia(df):
    df_cei = df[df['localizacao'] == "Ceilândia"]

    if df_cei.empty:
        print("\n[!] Nenhum registro encontrado para Ceilândia.")
        return

    print("\n--- Perfil Regional: Ceilândia ---\n")
    idade_media = int(df_cei['idade'].mean())
    renda_media = int(df_cei['renda_mensal'].mean())
    imovel_mais_desejado = df_cei['tipo_imovel_desejado'].mode()[0]
    faixa_etaria_predominante = df_cei['perfil_faixa_etaria'].mode()[0]

    nome_ficticio = fake.name()

    print(f"🏙️ Região: Ceilândia")
    print(f"   Nome fictício: {nome_ficticio}")
    print(f"   Faixa etária predominante: {faixa_etaria_predominante.replace('_', ' ').title()}")
    print(f"   Idade média: {idade_media} anos")
    print(f"   Renda média: R$ {renda_media}")
    print(f"   Tipo de imóvel mais desejado: {imovel_mais_desejado}")
    print("-" * 50)


gerar_bonecos_por_perfil(df)
perfil_ceilandia(df)



--- Bonecos dos Perfis Gerais ---

👤 Perfil: Adulto
   Nome: Helena Martins
   Idade média: 44 anos
   Renda média: R$ 9532
   Tipo de imóvel desejado: Apartamento
   Região predominante: Lago Sul
--------------------------------------------------
👤 Perfil: Jovem Adulto
   Nome: Ravi Lucca Freitas
   Idade média: 23 anos
   Renda média: R$ 4518
   Tipo de imóvel desejado: Kitnet
   Região predominante: Asa Sul
--------------------------------------------------
👤 Perfil: Idoso
   Nome: Zoe da Rosa
   Idade média: 69 anos
   Renda média: R$ 5743
   Tipo de imóvel desejado: Apartamento
   Região predominante: Guará
--------------------------------------------------

--- Perfil Regional: Ceilândia ---

🏙️ Região: Ceilândia
   Nome fictício: Clara da Mota
   Faixa etária predominante: Adulto
   Idade média: 46 anos
   Renda média: R$ 7293
   Tipo de imóvel mais desejado: Apartamento
--------------------------------------------------
